In [ ]:
!pip install wandb
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import wandb
import torch
wandb.login()

run = wandb.init(project="Email_subject_gen", job_type="model_loading")

#run = wandb.init(project="enron_subgen_gpt2", job_type="model_loading")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adarshj322. Use `wandb login --relogin` to force relogin


In [ ]:
artifact = run.use_artifact('Email_subject_gen/final_model:v0')

#artifact = run.use_artifact('enron-subgen-gpt2/model-1hhufzjv:v0')
# Download the artifact to a directory
artifact_dir = artifact.download()


wandb: Downloading large artifact final_model:v0, 299.04MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:9.7


In [ ]:
print(artifact_dir)

/content/artifacts/model-1hhufzjv:v0


In [ ]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
from peft import PeftModel

In [ ]:
#tokenizer= GPT2Tokenizer.from_pretrained(artifact_dir)

MODEL_KEY = 'olm/olm-gpt2-dec-2022'
tokenizer= GPT2Tokenizer.from_pretrained(MODEL_KEY)
tokenizer.add_special_tokens({'pad_token':'{PAD}'})

model = GPT2LMHeadModel.from_pretrained(MODEL_KEY)
model.resize_token_embeddings(len(tokenizer))
model.config.dropout = 0.1  # Set dropout rate
model.config.attention_dropout = 0.1

model = PeftModel.from_pretrained(model, artifact_dir)



tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/805k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/463k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
import re

In [ ]:
def clean_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
def generateSubject(email):
    tokenizer.padding_side='left'
    prompts_batch_ids = tokenizer(prompts_batch,
            padding=True, truncation=True, return_tensors='pt').to(model.device)
    output_ids = model.generate(
            **prompts_batch_ids, max_new_tokens=10,
            pad_token_id=tokenizer.pad_token_id)
    outputs_batch = [seq.split('<subject>')[1] for seq in
            tokenizer.batch_decode(output_ids, skip_special_tokens=True)]
    tokenizer.padding_side='right'

# Decode the output to get the predicted email subject
    predicted_subject = tokenizer.decode(output[0], skip_special_tokens=True)
    return predicted_subject





In [ ]:
text = "Dear Manager, I am writing to request a leave of absence for medical reasons from March 1 to March 10. Thank you for your understanding. Best, John Doe."

email = "<email>" + clean_text(text) + "<subject>"

print(email)

generateSubject(email)

<email>dear manager i am writing to request a leave of absence for medical reasons from march 1 to march 10 thank you for your understanding best john doe<subject>


TypeError: ignored

# this is for gpt2

In [ ]:
model = GPT2LMHeadModel.from_pretrained(artifact_dir)
#tokenizer= GPT2Tokenizer.from_pretrained(artifact_dir)
MODEL_KEY = 'gpt2'
tokenizer= GPT2Tokenizer.from_pretrained(MODEL_KEY)
tokenizer.add_special_tokens({'pad_token':'{PAD}'})

model.resize_token_embeddings(len(tokenizer))
model.config.dropout = 0.1  # Set dropout rate
model.config.attention_dropout = 0.1




RuntimeError: ignored

In [ ]:
!pip install evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7f6f4bdddacfb7efbc9af3cf5e5c445c1340542dcdbf25474bc2934996721afd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import pandas as pd

In [ ]:
import math
import evaluate

TEST_BATCH_SIZE = 16

bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

def test_data():
    prompts = list()
    references = list()
    subjects = list()
    ann0s = list()
    ann1s = list()
    ann2s = list()
    with open('dev.txt', 'r') as test:
      for sequence in test:
          sequence = sequence.strip()
          try:
              prompt, ref_text = sequence.split('<subject>')
              prompt += '<subject>'
              subject, subject_right = ref_text.split('<ann0>')
              ann0, ann0_right = subject_right.split('<ann1>')
              ann1, ann2 = ann0_right.split('<ann2>')
              if len(prompt) < 512 and subject != "" and ann0 != "" and ann1 != "" and ann2 != "":
                prompts.append(prompt)
                references.append([subject, ann0, ann1, ann2])
                subjects.append(subject)
                ann0s.append(ann0)
                ann1s.append(ann1)
                ann2s.append(ann2)
          except ValueError:
              continue

    tokenizer.padding_side='left'
    n_batches = math.ceil(len(prompts)*1.0 / TEST_BATCH_SIZE)
    outputs = list()
    for i in range(n_batches):
        prompts_batch = prompts[i*TEST_BATCH_SIZE : (i+1)*TEST_BATCH_SIZE]
        prompts_batch_ids = tokenizer(prompts_batch,
            padding=True, truncation=True, return_tensors='pt').to(model.device)
        output_ids = model.generate(
            **prompts_batch_ids, max_new_tokens=10,
            pad_token_id=tokenizer.pad_token_id)
        outputs_batch = [seq.split('<subject>')[1] for seq in
            tokenizer.batch_decode(output_ids, skip_special_tokens=True)]
        outputs.extend(outputs_batch)
        print(outputs_batch)
        print("------------------------")
    tokenizer.padding_side='right'
    #print(outputs)
    # with open('output.txt', 'w') as test:
    #   for item in outputs:
    #       test.write(item + '\n')

    df = pd.DataFrame({
        'prompt':prompts,
        'subject':subjects,
        'ann0':ann0s,
        'ann1':ann1s,
        'ann2':ann2s,
        'model_outputs':outputs
    })

    df.to_csv('validate_output.csv', index=False)

    bleu_score = bleu.compute(predictions=outputs, references=references)
    rouge_score = rouge.compute(predictions=outputs, references=references)

    return {
        'BLEU': round(bleu_score['bleu'], 4) * 100,
        'R1': round(rouge_score['rouge1'], 4) * 100,
        'R2': round(rouge_score['rouge2'], 4) * 100,
        'RL': round(rouge_score['rougeL'], 4) * 100,
        'RLsum': round(rouge_score['rougeLsum'], 4) * 100,
        #'meteor': round(meteor_score['meteor'], 4) * 100,

        #'METEOR': round(meteor_score['meteor'], 4) * 100
        }



In [ ]:
test_data()

['ena s session wednesday 10 00 august 8', '12 14 stepladder for pruning crepe my', 'new model for transport deals from yesterday and t', 'ft central physical file change ft central physical file change', 'delta flight 288 from houston to h', 'game 6 30 7 30 tonight 6 30 7 30', 'happy birthday old man i am not sure if your', 'emw line and settlements transfer to emw line', 'sitara for the cms deal', 'happy friday paul i hope you had a', ' interviews for the above positions in ', 'holiday party with good friends sara shackleton', 'draft of the csa for ', 'no donation is too small a donation is always', 'hey there hope you had a great holiday weekend', 'western resources inc ca revised ca for dynegy']
------------------------
[' invoices for gas and power 2001 2002 2002 2002 2002', 'city of houston option to preserve option to fix', 'westinghouse issues and the need', 'abb facility agreement form draft doc', 'policy update on md september 11 2001', 'rev 2 comparwrite redline for fuel cel

{'BLEU': 15.68,
 'R1': 41.349999999999994,
 'R2': 23.49,
 'RL': 38.61,
 'RLsum': 38.61}

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import ast
import evaluate


In [ ]:

def metric(data):
    #sub,ann0,ann1,ann2,gen_subject  = ["subject","ann0","ann1","ann2","gen_subject"]
    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")
    score = defaultdict(int)

    for _,x in tqdm(data.iterrows(),total=len(data)):

        #ref = ast.literal_eval(x["references"])
        # print(x["model_outputs"])
        # print("-----------------")



        rouge_s = rouge.compute(predictions=[x["model_outputs"]],references=[x["subjects"]])
        rouge_h = rouge.compute(predictions=[x["model_outputs"]] ,references=[[x["ann0"],x["ann1"],x["ann2"]]])

        score["rouge_1_sub"] += rouge_s['rouge1']
        score["rouge_2_sub"] += rouge_s['rouge2']
        score["rouge_L_sub"] += rouge_s['rougeL']

        score["rouge_1_human"] += rouge_h['rouge1']
        score["rouge_2_human"] += rouge_h['rouge2']
        score["rouge_L_human"] += rouge_h['rougeL']


        meteor_s = meteor.compute(predictions=[x["model_outputs"]],references=[x["subjects"]])
        meteor_h = meteor.compute(predictions=[x["model_outputs"]] ,references=[[x["ann0"], x["ann1"],x["ann2"]]])

        score["meteor_sub"] += meteor_s["meteor"]
        score["meteor_human"] += meteor_h["meteor"]

    print("The evalution scores are as follows:")

    for name,val in score.items():
        print(f"{name}: {val/len(data)}")

In [ ]:
metric(pd.read_csv("test_output.csv"))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 960/960 [05:16<00:00,  3.03it/s]

The evalution scores are as follows:
rouge_1_sub: 0.14980602641109633
rouge_2_sub: 0.05195092097230252
rouge_L_sub: 0.14451708910929145
rouge_1_human: 0.31420651345307943
rouge_2_human: 0.16652469785403248
rouge_L_human: 0.2977943447269111
meteor_sub: 0.12456784899288503
meteor_human: 0.2643787275019658
